# ML Project: eCommerce Inventory Prescription

In [ ]:
using JuMP
using Gurobi
using CSV
using DataFrames;

In [ ]:
# placeholders
items_tot = 20 # 20 items
time_tot = 52 # 52 weeks in a year

#### decision vars ####
x[i,t] --> inventory order from China of product i at time t
s[i,t] --> inventory available for sale (at AMZ warehouse) for product i at time t

#### params ####
d[i,t] --> demand for product i at time t
r[i] --> sales price for product i
c[i] --> manufacturing cost for product i
tr[i] --> transportation cost for product i

we'll incorporate volume later - for initial, lets assume fixed unit costs
v[i] --> volume (size) of product i
fba --> AMZ storage fee: $19/cbm per month
m[t] --> capital (money) available to purchase inventory at time t

revenue = 0.70 * sum( d[i,t]*r[i] for i=1:items_tot, t=1:time_tot ) --> amazon takes 30% cut
cost = sum( x[i,t]*(c[i]+tr[i]) for i=1:items_tot, t=1:time_tot ) + sum( fba*s[i,t] for i=1:items_tot, t=1:time_tot:4 )
    --> manu + transport cost of orders + monthly inventory fee for whatever inventory we have


In [ ]:
fba = 19

model = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attribute(model, "OutputFlag", 0)

@variable(model, x[i=1:items_tot, t=1:time_tot] >= 0, Int)
@variable(model, s[i=1:items_tot, t=1:time_tot] >= 0, Int)
@variable(model, m[t=1:time_tot] >= 0)

# capital is available 2 weeks after a product's sale
@constraint(model, [t=1:(time_tot-2)], m[t+2] == 0.7*sum(d[i,t]*r[i] for i=1:items_tot))
# the cost if inventory orders at time t must be less than our available capital
@constraint(model, [t=1:time_tot], sum(x[i,t]*(c[i]+tr[i]) for i=1:items_tot) <= m[t])
# sales at time t reduce inventory at warehouse immediately
@constraint(model, [i=1:items_tot, t=1:time_tot], s[i,t] == s[i,t] - d[i,t])
# inventory orders at time t are available for sale (arrive at warehouse) 8 weeks later
@constraint(model, [i=1:items_tot, t=1:time_tot], s[i,t+8] == s[i,t+8] + x[i,t])

@objective(model, Max, 0.7*sum(d[i,t]*r[i] for i=1:items_tot, for t=1:time_tot)
                        - sum(x[i,t]*(c[i]+tr[i]) for i=1:items_tot)
                        - fba*sum(s[i,t] for i=1:items_tot, t=(4*(0:12) + 1))
            )